# Performance Analysis: KSS/MFG & FA/MRA

Acronyms:
* KSS: Kevin Sanjaya Sukamuljo
* MFG: Marcus Fernaldi Gideon
* YW: Yuta Watanabe
* HE: Hiroyuki Endo

## Business Questions

- why were KSS/MFG not as dominant? is there a play style change? or drop in individual performance level?
- YW/HE are the only pair that could constantly beat KSS/MFG. why? what weaknesses did they exploit?

## General Stats

Ideas:

minions h2h vs rivals trend per meeting:
- dominance chart $\rightarrow$ win loss difference, points for vs points against difference
- li/liu, kam/son, endo/wat, aaron/soh, lee/wang, shetty/rankireddy, hoki/kobay, liu/ou, liang/wang, astrup/rasmussen, etc.

### Minions Performance Trend

#### Scrape Data

In [ ]:
# Web scrape fansite to get points, tournaments, ranking history. use selenium to iterate through weeks dropdown list, find kss & mfg points and scrape using beautifulsoup
# https://bwf.tournamentsoftware.com/ranking/ranking.aspx?rid=70

import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

player_id = 80057

history = pd.DataFrame(columns=['week', 'rank', 'points', 'tournaments'])

# open men's doubles rankings page
driver = webdriver.Chrome()
driver.get("https://bwf.tournamentsoftware.com/ranking")
driver.find_element(By.LINK_TEXT, "BWF World Rankings").click()
driver.find_element(By.LINK_TEXT, "Men's Doubles").click()
wait = WebDriverWait(driver, 10)

# week options
week_element = driver.find_element(By.ID,"cphPage_cphPage_cphPage_dlPublication")
select_week = Select(week_element)

for index in range(len(select_week.options)):
    # wait until select week element is loaded
    wait.until(EC.presence_of_element_located((By.ID, "cphPage_cphPage_cphPage_dlPublication")))    
    week_element = driver.find_element(By.ID,"cphPage_cphPage_cphPage_dlPublication")
    
    # unhide week selector
    driver.execute_script("arguments[0].style.removeProperty('display');", week_element)
    
    # change week
    select_week = Select(week_element)
    week = select_week.options[index].get_attribute("innerHTML") # get week
    select_week.select_by_index(index)
    
    # wait until results per page element is loaded
    wait.until(EC.presence_of_element_located((By.ID, "_pagesize")))
    
    # change results per page
    select_results_per_page = Select(driver.find_element(By.ID,"_pagesize"))
    select_results_per_page.select_by_index(3)
    
    # find player id
    while True:
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, f"//*[text()[contains(., '{player_id}')]]//..")))
            
        except:
            try:
                driver.find_element(By.CLASS_NAME, "page_next").click()
                
            except:
                # if not in rankings, insert nans
                rank = np.nan
                points = np.nan
                tournaments = np.nan
                break
        else:
            pair = driver.find_element(By.XPATH, f"//*[text()[contains(., '{player_id}')]]//..")
            rank = pair.find_element(By.XPATH, ".//*[contains(@class, 'rank')][1]//div").get_attribute("innerHTML")
            points = pair.find_element(By.XPATH, ".//*[contains(@class, 'right rankingpoints')][1]").get_attribute("innerHTML")
            tournaments = pair.find_element(By.XPATH, ".//*[contains(@class, 'right')][2]").get_attribute("innerHTML")
            break

    # save data to dataframe
    history.at[index, 'week'] = week
    history.at[index, 'rank'] = rank
    history.at[index, 'points'] = points
    history.at[index, 'tournaments'] = tournaments

    time.sleep(2)
    
history.to_csv("ranking-history.csv", index=False)

In [2]:
# convert datatypes
from datetime import datetime
import pandas as pd

history = pd.read_csv('ranking-history.csv')

date_format = "%m/%d/%Y"
dates = []

for date_string in history['week']:
    week = datetime.strptime(date_string, date_format).date()
    dates.append(week)

history['week'] = dates
history['rank'] = pd.to_numeric(history['rank'], errors = 'coerce')
history['points'] = pd.to_numeric(history['points'], errors = 'coerce')
history['tournaments'] = pd.to_numeric(history['tournaments'], errors = 'coerce')

history.head(5)

,week,rank,points,tournaments
0,2024-07-02,200.0,6350.0,2.0
1,2024-06-25,199.0,6350.0,2.0
2,2024-06-18,202.0,6350.0,2.0
3,2024-06-11,200.0,6350.0,2.0
4,2024-06-04,203.0,6350.0,2.0


In [31]:
# get player wikipedia page
from bs4 import BeautifulSoup
import requests

page = requests.get("https://en.wikipedia.org/wiki/Kevin_Sanjaya_Sukamuljo")
soup = BeautifulSoup(page.content, "html.parser")
tables = soup.find_all("table", class_ = "wikitable")

# get BWF tournaments results
rows = tables[18].find_all("tr") 
years = [year.text.strip() for year in rows[1].find_all("th")]

tournament_results = pd.DataFrame()

tournament_results['year'] = years

for row in rows[2:30]: #iterate through tournament rows
    columns = row.find_all("td")
    tournament_name = columns[0].text.strip()
    results = []
    
    for column in columns[1:-2]: # iterate through results (columns)
        if 'colspan' in column.attrs: # consecutive results 
            for colspan in range(int(column.attrs['colspan'])):
                results.append(column.text.strip())
        else:
            results.append(column.text.strip())
    
    if len(results) > 0:
        tournament_results[tournament_name] = results
        
tournament_results.to_csv('tournament-results.csv', index=False)

In [4]:
# generate performance trend
import pandas as pd

tournament_results = pd.read_csv('tournament-results.csv')

did_not_participate = ['A', 'NH', 'N/A', 'w/d', 'DNQ', 'NA','']

columns = tournament_results.columns[1:] # list of tournaments

no_of_tournaments = []
avg_points_earned_percentage = []
total_top_four = []
total_finals = []
total_titles = []

for year in tournament_results['year']: #iterate through years
    tournament_count = 0
    points_earned_percentage = 0
    top_four = 0
    finals = 0
    titles = 0
    
    for column in columns:  #iterate through tournaments
        result = tournament_results.loc[tournament_results['year'] == year][column].iloc[0] # tournament result
        
        if result not in did_not_participate:
            tournament_count += 1
            
            if result == '1R':
                points_earned_percentage += 0.25
            elif result == '2R':
                points_earned_percentage += 0.4
            elif result == 'QF':
                points_earned_percentage += 0.55
            elif result == 'SF':
                points_earned_percentage += 0.7
                top_four += 1
            elif result == 'F':
                points_earned_percentage += 0.85
                top_four += 1
                finals += 1
            elif result == 'W':
                points_earned_percentage += 1
                top_four += 1
                finals += 1
                titles += 1
    
    no_of_tournaments.append(tournament_count)
    avg_points_earned_percentage.append(points_earned_percentage/tournament_count)
    total_top_four.append(top_four)
    total_finals.append(finals)
    total_titles.append(titles)
    
performance_trend = pd.DataFrame()

performance_trend['year'] = tournament_results['year'].astype(int)
performance_trend['no_of_tournaments'] = no_of_tournaments
performance_trend['titles'] = total_titles
performance_trend['finals'] = total_finals
performance_trend['top_four'] = total_top_four
performance_trend['avg_points_percentage_per_tournament'] = avg_points_earned_percentage
        

In [5]:
# end of year rankings & points
import numpy as np

history['week'] = pd.to_datetime(history['week'], errors='coerce')

years = np.unique(history['week'].dt.year)
end_of_year_rank = []
end_of_year_points = []
end_of_year_tournaments = []

for year in years:
    # get data from last day of year
    last_day_of_year = history.loc[history['week'].dt.year == year].sort_values(by='week', ascending=False).reset_index(drop=True)
    end_of_year_rank.append(last_day_of_year['rank'][0])
    end_of_year_points.append(last_day_of_year['points'][0])
    end_of_year_tournaments.append(last_day_of_year['tournaments'][0])

# join to performance trend table
end_of_year_results = pd.DataFrame()
end_of_year_results['year'] = years
end_of_year_results['end_of_year_rank'] = end_of_year_rank
end_of_year_results['end_of_year_points'] = end_of_year_points
end_of_year_results['tournaments_included_in_rank'] = end_of_year_tournaments

performance_trend = pd.merge(performance_trend, end_of_year_results, how='outer', on='year')

#### When did the minions' results start to drop?

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(go.Scatter(
    x=history['week'] , 
    y=history['rank'], 
    mode='lines', 
    line_color='blue',
    name="points"),
    secondary_y=False,
)

fig.update_yaxes(
    title_text="Rank",
    range=[100,-5], 
    secondary_y=False
)

# fig.add_trace(go.Scatter(
#     x=history['week'] , 
#     y=history['tournaments'], 
#     mode='lines', 
#     line_color='green',
#     name="tournaments"),
#     secondary_y=True,
# )

# fig.update_yaxes(
#     title_text="Tournaments", 
#     secondary_y=True
#     # autorange="reversed"
# )

# Add figure title
fig.update_layout(
    title_text="KSS MFG BWF Ranking & Points History",
    height = 600
)

# Set x-axis title
fig.update_xaxes(title_text="Year")

# set y-axis ticks
fig.update_yaxes(tickvals = [1] + list(range(10,100,10)))


fig.show()

Data source: https://bwf.tournamentsoftware.com/ranking/ranking.aspx?rid=70

Performance trend only shows drastic drop in 2023. Due to the ranking freeze applied by BWF from march 2020 to feb 2021 due to covid, KSS/MFG actually retained a lot of points that they earned before covid, so this does not reflect their true results.

Need to get total points earned from each tournament to compare the max no. of points possible vs actual received. This should be a better reflection of their actual performance trend. 

In [6]:
# plot yearly performance trend
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Main y-axis
fig.add_trace(go.Scatter(
        x=performance_trend['year'], 
        y=round(performance_trend['avg_points_percentage_per_tournament']*100,2), 
        mode='lines+markers+text',
        line_color='blue', 
        text=performance_trend['no_of_tournaments'],
        texttemplate = "%{y}% <br>(%{text})",
        textposition="top center",
        name="points percentage per tournament"
    ),
    secondary_y=False,
)

fig.update_yaxes(
    title_text="Avg Percentage <br> (No. of Tournaments)", 
    secondary_y=False
    # autorange="reversed"
)

# Add figure title
fig.update_layout(
    title_text="KSS/MFG Average Points Earned per Tournament in BWF Super Series/World Tour (Percentage)",
    height=600
)

# Set x-axis title
fig.update_xaxes(
    title_text="Year",
    tickvals = performance_trend['year']
)

# set y-axis range
fig.update_yaxes(
    range=[round(performance_trend['avg_points_percentage_per_tournament']*100,2).min() - 5, round(performance_trend['avg_points_percentage_per_tournament']*100,2).max() + 10],
    tickvals = list(range(0,100,10))
)

fig.show()

Data source: https://en.wikipedia.org/wiki/Kevin_Sanjaya_Sukamuljo

Graph shows percentage of points that the minions actually earned from the maximum possible amount of points that can be earned. This only takes into account tournaments where KSS/MFG played at least one match.

Note that although each tournament has different amount of points up for grabs depending on its level (super 1000, 750, 500, and so on), the points ratio generally remains the same. E.g. winning the tournament gets you 100% of the max amount of points, 2nd place gets you 85% of the max amount of points

KSS/MFG peaked in 2017 and stayed relatively consistent up to 2019, but showed a downward trend from 2020 onwards. 2020 was an anomaly, as they only played in 3 tournaments due to MFG's injury and the limited amount of tournaments available due to covid restrictions.



In [ ]:
# plot titles, finals, top fours
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(
        name='No. of Tournaments', 
        x=performance_trend['year'], 
        y=performance_trend['no_of_tournaments'],
        texttemplate='%{y}'
    ),
    go.Bar(
        name='> Semi-finals', 
        x=performance_trend['year'], 
        y=performance_trend['top_four'],
        texttemplate='%{y}'
    ),
    go.Bar(
        name='> Finals', 
        x=performance_trend['year'], 
        y=performance_trend['finals'],
        texttemplate='%{y}'
    ),
    go.Bar(
        name='Wins', 
        x=performance_trend['year'], 
        y=performance_trend['titles'],
        texttemplate='%{y}'
    )
])
# Change the bar mode
fig.update_layout(barmode='group')

# Add figure title
fig.update_layout(
    title_text="KSS MFG Top Four or Better Finishes",
    height=500
)

fig.update_xaxes(
    title_text="Year",
    tickvals = performance_trend['year']
)

fig.update_yaxes(title_text="Total")


fig.show()

Data source: https://en.wikipedia.org/wiki/Kevin_Sanjaya_Sukamuljo

This graph reinforces the fact that KSS/MFG's performance started to drop in 2020. before that, they were consistently performing at a high level for 3 years (2017-2019), reaching at least the semi-finals on 10 occasions each year.


## Match Insights

- who is winning the initiative from service situation? --> server & receiver, serve position (odd/even point), who won initiative, serve type (low T, low body, low wide, flick T, flick wide), return type (drop, drive, clear), return destination (front left, mid right, back middle)
- does winning initiative lead to winning point? --> compare with rallys won percentage to show importance of initiative
- most effective point winning shots? --> timestamp, type of stroke (smash, drop, drive, net, ...), where it was played from & played to (front left, mid right, back middle), who played it & received it
- opponent weaknesses? shots leading to weak reply --> same as above, weak reply are shots with a high chance of losing point (e.g. weak lifts) 
- opponent weaknesses? forced errors --> who made error, shot type faced (smash, drop, etc.), shot placement (high forehand, low backhand), shot origin & destination (front left, back right, ...), erroneous shot type, erroneous shot origin & destination
- ~~no of short rallies~~ 
- ~~momentum change --> plot points vs rally no~~
- no of winners vs errors & unforced errors by person
- shots from above net vs shots from below net
- ~~how involved in the match $\rightarrow$ percentage shots played vs total shots played. any players isolated?~~
- how effective is back player from the back court $\rightarrow$ shots leading to weak replies/interceptions, winners vs errors from the back
- how effective is front player at the net $\rightarrow$ net duels won/lost, winners/errors at the net, percentage shots played at the net vs elsewhere (shots heatmap)

### KSS/MFG vs YW/HE BAC 2019 Final - Match Analysis 

### - Why did KSS/MFG lose so badly? 
### - What did YW/HE do right? 
### - How can KSS/MFG win against YW/HE?

#### Gather Data

In [ ]:
# process shots data
import pandas as pd

shots_game_1_half_1 = pd.read_csv('BAC 2019 - KSS MFG v YW HE - Game 1 1st Half.csv', sep=',')
shots_game_1_half_2 = pd.read_csv('BAC 2019 - KSS MFG v YW HE - Game 1 2nd Half.csv', sep=',')

shots_game_1 = pd.concat([shots_game_1_half_1, shots_game_1_half_2]).reset_index(drop=True)

In [ ]:
# separate player & stroke type
player_name = []
stroke = []

for player in shots_game_1['Player']:
    split = player.split(' ')
    
    player_name.append(split[0])
    stroke.append(split[1])
    
shots_game_1['Player'] = player_name
shots_game_1['Stroke'] = stroke

In [ ]:
# add score, rally number, rallies dataframe

# index of serves in shots dataframe
serves = shots_game_1.loc[shots_game_1['Event'].str.contains('Serve')]
serves_index = list(serves.index)

# last shots from each rally
last_shots_index = list(serves.index-1)[1:]
last_shots_index.append(shots_game_1.index[-1])
last_shots = shots_game_1.loc[last_shots_index]

# declare rally dataframe
teams = shots_game_1['Team'].unique()
column_names = teams + '_score'
rallies_game_1 = pd.DataFrame()

#declare arrays to insert to dataframe
rally_no_array = [0]
rally_length_array = [0]
rally_winner_array = ['-']
score_team_0_array = [0]
score_team_1_array = [0]

# initialize variables
rally_no = 1
score_team_0 = 0
score_team_1 = 0

for index, row in last_shots.iterrows():
    
    # add score columns to shots dataframe
    shots_game_1.loc[serves_index[rally_no-1]:last_shots_index[rally_no-1], column_names[0]] = score_team_0
    shots_game_1.loc[serves_index[rally_no-1]:last_shots_index[rally_no-1], column_names[1]] = score_team_1
    shots_game_1.loc[serves_index[rally_no-1]:last_shots_index[rally_no-1], 'rally_no'] = rally_no
    
    # determine point winner
    if row['X2'] == '-': # error
        if row['Team'] == teams[0]: # point for other team
            score_team_1 += 1
            rally_winner = teams[1]
        else:
            score_team_0 += 1
            rally_winner = teams[0]
            
    else: # winner
        if row['Team'] == teams[0]:
            score_team_0 += 1
            rally_winner = teams[0]
        else:
            score_team_1 += 1
            rally_winner = teams[1]
    
    # record score
    rally_no_array.append(rally_no)
    rally_length_array.append(last_shots_index[rally_no-1] - serves_index[rally_no-1] + 1)
    rally_winner_array.append(rally_winner)
    score_team_0_array.append(score_team_0)
    score_team_1_array.append(score_team_1)
    
    rally_no += 1

rallies_game_1['rally_no'] = rally_no_array
rallies_game_1['rally_length'] = rally_length_array
rallies_game_1['rally_winner'] = rally_winner_array
rallies_game_1[column_names[0]] = score_team_0_array
rallies_game_1[column_names[1]] = score_team_1_array
rallies_game_1['score_delta'] = rallies_game_1[column_names[0]] - rallies_game_1[column_names[1]]

In [ ]:
%store shots_game_1
%store rallies_game_1

In [2]:
%store -r shots_game_1
%store -r rallies_game_1

In [3]:
# separate data by player
teams = shots_game_1['Team'].unique()

players_team_0 = shots_game_1['Player'].loc[shots_game_1['Team'] == teams[0]].unique()
players_team_1 = shots_game_1['Player'].loc[shots_game_1['Team'] == teams[1]].unique()

shots_team_0_player_0 = shots_game_1.loc[(~shots_game_1['Event'].str.contains('Serve')) & (shots_game_1['Player'] == players_team_0[0])]
shots_team_0_player_1 = shots_game_1.loc[(~shots_game_1['Event'].str.contains('Serve')) & (shots_game_1['Player'] == players_team_0[1])]
shots_team_1_player_0 = shots_game_1.loc[(~shots_game_1['Event'].str.contains('Serve')) & (shots_game_1['Player'] == players_team_1[0])]
shots_team_1_player_1 = shots_game_1.loc[(~shots_game_1['Event'].str.contains('Serve')) & (shots_game_1['Player'] == players_team_1[1])]

#### How did the momentum shift during the game?

In [ ]:
# plot scores
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Team 0
fig.add_trace(go.Scatter(
        x=rallies_game_1['rally_no'], 
        y=rallies_game_1[teams[0] + '_score'], 
        mode='lines+markers',
        line_color='blue', 
        # text=performance_trend['no_of_tournaments'],
        # texttemplate = "%{y}% <br>(%{text})",
        # textposition="top center",
        name= teams[0]
    ),
    secondary_y=False,
)

# Team 1
fig.add_trace(go.Scatter(
        x=rallies_game_1['rally_no'], 
        y=rallies_game_1[teams[1] + '_score'], 
        mode='lines+markers',
        line_color='red', 
        # text=performance_trend['no_of_tournaments'],
        # texttemplate = "%{y}% <br>(%{text})",
        # textposition="top center",
        name= teams[1]
    ),
    secondary_y=False,
)

fig.update_yaxes(
    title_text="Score", 
    secondary_y=False
    # autorange="reversed"
)

fig.add_hline(y=11, line_dash="dash",
              annotation_text="Interval", 
              annotation_position="bottom right")

# Add figure title
fig.update_layout(
    title_text="Points Tally - Game 1",
    height=600
)

# Set x-axis title
fig.update_xaxes(
    title_text="Rally No.",
    tickvals = rallies_game_1['rally_no']
)

# set y-axis range
fig.update_yaxes(
    # range=[round(performance_trend['avg_points_percentage_per_tournament']*100,2).min() - 5, round(performance_trend['avg_points_percentage_per_tournament']*100,2).max() + 10],
    tickvals = list(range(0,rallies_game_1[[teams[0] + '_score', teams[1] + '_score']].max().max() + 1,1))
)

fig.show()

YW/HE got the better start, but KSS/MFG caught up and led by 1 point at the interval. The 2nd half was also tight, but YW/HE crucially won 3 consecutive points from 18 all. **Find out what caused INA to lose the last 3 points!**

#### Who was the player most involved in the game on each team?

In [ ]:
# player involvement
import plotly.graph_objects as go
from plotly.subplots import make_subplots

involvement_team_0 = [
    len(shots_team_0_player_0), 
    len(shots_team_0_player_1)
]
involvement_team_1 = [
    len(shots_team_1_player_0), 
    len(shots_team_1_player_1)
]

# plots
fig = make_subplots(
    rows=1, 
    cols=2,
    # row_heights=[0.5, 0.25, 0.25],
    subplot_titles=(teams[0], 
                    teams[1]
                    ),
    specs=[
        [{"type": "pie"}, {"type": "pie"}]
    ]
    )

# player involvement team 0
fig.add_trace(
    go.Pie(
        labels=players_team_0, 
        values=involvement_team_0,
        textinfo='label+percent+value',
        # marker=dict(colors=['orangered', 'cornflowerblue'])
    ),
    row=1, 
    col=1
)

# player involvement team 1
fig.add_trace(
    go.Pie(
        labels=players_team_1, 
        values=involvement_team_1,
        textinfo='label+percent+value',
        # marker=dict(colors=['orangered', 'cornflowerblue'])
    ),
    row=1, 
    col=2
)

# Add figure title
fig.update_layout(
    title_text="Player Involvement - Game 1",
    height=500
)

fig.show()

Player involvement of both players from both teams were pretty much equal, with KSS slightly more involved than MFG.

#### What are each player's go to shot type?

In [ ]:
# most played shot type by each player
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# data
shots_frequency_team_0_player_0 = shots_team_0_player_0['Event'].value_counts()
shots_frequency_team_0_player_1 = shots_team_0_player_1['Event'].value_counts()
shots_frequency_team_1_player_0 = shots_team_1_player_0['Event'].value_counts()
shots_frequency_team_1_player_1 = shots_team_1_player_1['Event'].value_counts()

# helper functions
def color(indexes):
    colors = []
    
    for index in list(indexes):
        if index == 'Slow Lift':
            colors.append('darkblue')
        elif index == 'Fast Lift':
            colors.append('blue')
        elif index == 'Drive Up':
            colors.append('green')
        elif index == 'Drive Flat':
            colors.append('yellowgreen')
        elif index == 'Drive Down':
            colors.append('greenyellow')
        elif index == 'Slow Drop':
            colors.append('orange')
        elif index == 'Fast Drop':
            colors.append('darkorange')
        elif index == 'Half Smash':
            colors.append('orangered')
        else:
            colors.append('red')
            
    return colors

# plots
fig = make_subplots(
    rows=2, 
    cols=2,
    # row_heights=[0.5, 0.25, 0.25],
    subplot_titles=( 
                    players_team_0[0], 
                    players_team_0[1], 
                    players_team_1[0], 
                    players_team_1[1]
                    ),
    specs=[
        [{"type": "pie"}, {"type": "pie"}],
        [{"type": "pie"},{"type": "pie"}]
    ]
)

# shot frequency team 0 player 0
fig.add_trace(
    go.Pie(
        labels=shots_frequency_team_0_player_0.index, 
        values=shots_frequency_team_0_player_0,
        textinfo='percent',
        marker=dict(colors=color(shots_frequency_team_0_player_0.index))
    ),
    row=1, 
    col=1
)

# shot frequency team 0 player 1
fig.add_trace(
    go.Pie(
        labels=shots_frequency_team_0_player_1.index, 
        values=shots_frequency_team_0_player_1,
        textinfo='percent',
        marker=dict(colors=color(shots_frequency_team_0_player_1.index))
    ),
    row=1, 
    col=2
)

# shot frequency team 1 player 0
fig.add_trace(
    go.Pie(
        labels=shots_frequency_team_1_player_0.index, 
        values=shots_frequency_team_1_player_0,
        textinfo='percent',
        marker=dict(colors=color(shots_frequency_team_1_player_0.index))
    ),
    row=2, 
    col=1
)

# shot frequency team 1 player 1
fig.add_trace(
    go.Pie(
        labels=shots_frequency_team_1_player_1.index, 
        values=shots_frequency_team_1_player_1,
        textinfo='percent',
        marker=dict(colors=color(shots_frequency_team_1_player_1.index))
    ),
    row=2, 
    col=2
)

# Add figure title
fig.update_layout(
    title_text="Shot Variation of Each Player - Game 1",
    height=800
)

fig.show()

JPN's shot variation is more defensive oriented, predominantly playing lifts and upwards or flat drives. 

YW is especially defensive oriented, with his 1/3 of his shots consisting of lifts. He plays a lot of **fast lifts**, most likely to move the opponent back court player and force a weaker attacking shot **to create counter opportunities**. **Look for fast lifts from YW that causes a turnover!**

HE also plays a lot of lifts (mostly slow), but he also plays a lot of **upwards drives**. Upwards drives can be quite risky to play, as the shuttle will most likely be above the net when the opponent receives it. This shot can easily be countered if the opponent is alert and can be a **potential weakness**. **Look for upwards drives from HE that causes JPN to lose points or at least be put under pressure!**

Meanwhile, INA's shot variation is more attacking oriented. 

MFG mixes up his attack with smashes and drop shots, but mostly plays smashes. Meanwhile, KSS predominantly plays drops instead of smashes. INA team mostly plays with a classic formation with KSS as the front court player and MFG playing at the back. This could be why KSS prefers to save his energy when at the back court, prioritizing placement shots instead of smashes. Instead, **MFG is the one who plays from the back and tries to force weak replies to create killing opportunities for KSS**. **Find out which shots from MFG are most effective in creating winners/weak replies!** 

Interesting to note that MFG plays a lot of slow lifts. These lifts are most likely played in defensive situations. Meanwhile, KSS rarely plays lifts. This could indicate that **INA team mainly relies on KSS to turn defensive situations into offense**, as slow lifts are less likely to result in a turn-around. **Compare turnovers from KSS & MFG** 

#### How many short & long rallies were there? Which team one more short rallies? Which team won more long rallies?

In [ ]:
# short & long rallies
import plotly.graph_objects as go
from plotly.subplots import make_subplots

threshold_short = 5
threshold_medium = 10
threshold_long = 15

# plots
fig = make_subplots(
    rows=3, 
    cols=2,
    # row_heights=[0.5, 0.25, 0.25],
    subplot_titles=( 
                    "Rally Lengths", 
                    "",
                    "Short Rallies Won", 
                    "Medium Rallies Won",
                    "Long Rallies Won",
                    "Very Long Rallies Won"
                    ),
    specs=[
        [{"type": "pie", "colspan": 2}, {}],
        [{"type": "pie"},{"type": "pie"}],
        [{"type": "pie"},{"type": "pie"}]
    ]
)

# amount of short & long rallies
fig.add_trace(
    go.Pie(
        labels=[
            '<=' + str(threshold_short) +  ' Shots (Short)', 
            str(threshold_short + 1) + '-' + str(threshold_medium) + ' Shots (Medium)',
            str(threshold_medium + 1) + '-' + str(threshold_long) + ' Shots (Long)',
            '>' + str(threshold_long) +  ' Shots (Very Long)' 
        ], 
        values=[
            len(rallies_game_1[1:].loc[rallies_game_1['rally_length'] <= threshold_short]), 
            len(rallies_game_1[1:].loc[[threshold_short < x <= threshold_medium for x in rallies_game_1[1:]['rally_length']]]),
            len(rallies_game_1[1:].loc[[threshold_medium < x <= threshold_long for x in rallies_game_1[1:]['rally_length']]]),
            len(rallies_game_1[1:].loc[rallies_game_1['rally_length'] > threshold_long])
        ],
        sort=True,
        textinfo='percent+value+label'
    ),
    row=1, 
    col=1
)

# short rallies won
fig.add_trace(
    go.Pie(
        labels=teams, 
        values=[
            len(rallies_game_1.loc[(rallies_game_1['rally_length'] <= threshold_short) & (rallies_game_1['rally_winner'] == teams[0])]),
            len(rallies_game_1.loc[(rallies_game_1['rally_length'] <= threshold_short) & (rallies_game_1['rally_winner'] == teams[1])])
        ],
        textinfo='percent+value+label'
    ),
    row=2, 
    col=1
)

# medium rallies won
fig.add_trace(
    go.Pie(
        labels=teams, 
        values=[
            len(rallies_game_1.loc[([threshold_short < x <= threshold_medium for x in rallies_game_1['rally_length']]) & (rallies_game_1['rally_winner'] == teams[0])]),
            len(rallies_game_1.loc[([threshold_short < x <= threshold_medium for x in rallies_game_1['rally_length']]) & (rallies_game_1['rally_winner'] == teams[1])])
        ],
        textinfo='percent+value+label'
    ),
    row=2, 
    col=2
)

# long rallies won
fig.add_trace(
    go.Pie(
        labels=teams, 
        values=[
            len(rallies_game_1.loc[([threshold_medium < x <= threshold_long for x in rallies_game_1['rally_length']]) & (rallies_game_1['rally_winner'] == teams[0])]),
            len(rallies_game_1.loc[([threshold_medium < x <= threshold_long for x in rallies_game_1['rally_length']]) & (rallies_game_1['rally_winner'] == teams[1])])
        ],
        textinfo='percent+value+label'
    ),
    row=3, 
    col=1
)

# very long rallies won
fig.add_trace(
    go.Pie(
        labels=teams, 
        values=[
            len(rallies_game_1.loc[(rallies_game_1['rally_length'] > threshold_long) & (rallies_game_1['rally_winner'] == teams[0])]),
            len(rallies_game_1.loc[(rallies_game_1['rally_length'] > threshold_long) & (rallies_game_1['rally_winner'] == teams[1])])
        ],
        textinfo='percent+value+label'
    ),
    row=3, 
    col=2
)

# Add figure title
fig.update_layout(
    title_text="Short vs. Long Rallies - Game 1",
    height=1000
)

fig.show()

KSS/MFG have the advantage in short and medium rallies. They managed to win almost 2/3 of all the short and medium rallies in the game. They are known to be strong in service situations, so they tend to win a lot of points in service situations (within the first 5 shots). If not, they would usually at least win the attacking advantage in the service situation, putting their opponent under pressure in the first 5 shots, and then win the point in the following couple of shots (within 6-10 shots in total).  

However, YW/HE frequently managed to survive the service situation and prolong the rally to more than 10 shots. And in these occasions, YW/HE dominated KSS/MFG. YW/HE managed to win more than 3/4 of the very long rallies.

KSS/MFG need to try and **win the point in 5 shots**. **Look for short & medium rallies that INA won. Find out whether winning the attacking advantage in the service situation leads to them winning the point. If so, find out how to win the service situation against YW/HE.** 

There will be cases where YW/HE survives the service situation. KSS/MFG need to have a solid game plan to win long rallies. **Look for long rallies where INA won. Find out how to win long rallies against YW/HE** 

**Compare with long rallies where KSS/MFG lost. What is different? One player isolated?**

#### Where does each player usually play from? Are there certain types of shots that are played in certain positions?  

In [4]:
# function to map coordinates from fc python to actual court coordinates

# court proportions on fc python do not follow standards: 
# e.g. ratio of the width of tram lines to the whole with of the court may differ. this can result in false plotting on the actual court. 

# map according to sections:
# for y coordinates: tram line left, middle of court, tram line right
# for x coordinates: rear service area 1 & 2, net area, mid court area 1 & 2

# fc python rugby court coordinates:
# pos, x, y
# top left outer tram, 19, 8
# top left inner tram, 19, 22
# bottom left inner tram, 19, 79
# bottom left outer tram, 19, 93
# top left outer back service line, 25, 8
# top left inner back service line, 25, 22
# bottom left inner back service line, 25, 79
# bottom left outer back service line, 25, 93
# top left outer front service line, 43, 8
# top right outer front service line, 57, 8
# top right outer back service line, 75, 8
# top right outer tram, 81, 8

def map_values(value, value_min, value_max, output_min, output_max):
    return ( ( (value - value_min) / (value_max - value_min) ) * (output_max - output_min) )  + output_min

def map_values_x(x): 
    if x == '-':
        x = 0
    else:
        x = int(x)
       
    # from left to right
    if 19 <= x < 25: # back court 1 
        return map_values(x, 19, 25, 0, 760)
    elif 25 <= x < 43: # mid court 1
        return map_values(x, 25, 43, 760, 6710-1980)
    elif 43 <= x < 57: # net area
        return map_values(x, 43, 57, 6710-1980, 6710+1980)
    elif 57 <= x < 75: # mid court 2
        return map_values(x, 57, 75, 6710+1980, 13410-760)
    elif 75 <= x <= 81: # back court 2
        return map_values(x, 75, 81, 13410-760, 13410)
    else:
        return None
    
def map_values_y(y):   
    if y == '-':
        y = 0
    else:
        y = int(y)
     
    # from left to right
    if 8 <= y < 22: # back court 1 
        return map_values(y, 8, 22, 0, 450)
    elif 22 <= y < 79: # mid court 1
        return map_values(y, 22, 79, 450, 6100-450)
    elif 79 <= y <= 93: # net area
        return map_values(y, 79, 93, 6100-450, 6100)
    else:
        return None

In [5]:
# function to draw badminton court
# adapted from: https://github.com/lukarh/assist-tracking-app/blob/master/dashboard-app.py

import plotly.graph_objects as go
import numpy as np

def draw_plotly_court(fig, fig_width=500, fig_height=870, margins=50, lwidth=3,
                      show_title=True, labelticks=True, show_axis=True,
                      glayer='below', bg_color='white'):

    fig.update_layout(height=870,
                      template='plotly_dark')

    # Set axes ranges
    court_x0 = 0
    court_x1 = 13410
    court_y0 = 6100 
    court_y1 = 0
    fig.update_xaxes(
        range=[court_x0 - margins, court_x1 + margins],
        visible=show_title
    )
    fig.update_yaxes(
        range=[court_y0 + margins, court_y1 -  margins],
        visible=show_title
    )

    main_line_col = "#777777"
    
    fig.update_layout(
        # Line Horizontal
        # margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor=bg_color,
        plot_bgcolor=bg_color,
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            # fixedrange=True,
            visible=show_axis,
            showticklabels=labelticks,
        ),
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            # fixedrange=True,
            visible=show_axis,
            showticklabels=labelticks,
        ),
        yaxis2=dict(
            scaleanchor="x2",
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            # fixedrange=True,
            visible=show_axis,
            showticklabels=labelticks,
            range=[court_y0 + margins, court_y1 -  margins]
        ),
        xaxis2=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            # fixedrange=True,
            visible=show_axis,
            showticklabels=labelticks,
            range=[court_y0 + margins, court_y1 -  margins]
        ),
        shapes=[
            # court border
            dict(
                type="rect", x0=court_x0, y0=court_y0, x1=court_x1, y1=court_y1,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            # tram lines
            dict(
                type="line", x0=0, y0=450, x1=13410, y1=450,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            dict(
                type="line", x0=0, y0=6100-450, x1=13410, y1=6100-450,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            # front service line
            dict(
                type="line", x0=6710-1980, y0=0, x1=6710-1980, y1=6100,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            dict(
                type="line", x0=6710+1980, y0=0, x1=6710+1980, y1=6100,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            # rear service line
            dict(
                type="line", x0=760, y0=0, x1=760, y1=6100,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            dict(
                type="line", x0=13410-760, y0=0, x1=13410-760, y1=6100,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            # middle line
            dict(
                type="line", x0=0, y0=3050, x1=6710-1980, y1=3050,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            dict(
                type="line", x0=6710+1980, y0=3050, x1=13410, y1=3050,
                line=dict(color=main_line_col, width=lwidth),
                layer=glayer
            ),
            # net
            dict(
                type="line", x0=6710, y0=0, x1=6710, y1=6100,
                line=dict(color=main_line_col, width=lwidth, dash="dash"),
                layer=glayer
            )
        ]
    )
    return True

In [6]:
# function to map colors to shot type
def assign_color_by_shot_type(indexes):
    colors = []
    
    for index in list(indexes):
        if index == 'Slow Lift':
            colors.append('darkblue')
        elif index == 'Fast Lift':
            colors.append('blue')
        elif index == 'Drive Up':
            colors.append('green')
        elif index == 'Drive Flat':
            colors.append('yellowgreen')
        elif index == 'Drive Down':
            colors.append('greenyellow')
        elif index == 'Slow Drop':
            colors.append('orange')
        elif index == 'Fast Drop':
            colors.append('darkorange')
        elif index == 'Half Smash':
            colors.append('orangered')
        else:
            colors.append('red')
            
    return colors

In [11]:
# plot shots
fig = go.Figure()

# draw badminton court
draw_plotly_court(fig, show_title=True, labelticks=False, show_axis=False,
                  glayer='above', bg_color='black', margins=200)

shots = shots_team_1_player_0

for shot_type in shots['Event'].unique(): # add trace for every shot type
    
    shots_by_type = shots.loc[shots['Event'] == shot_type]
    
    # convert coordinates
    origin_x = list(map(map_values_x, shots_by_type['X']))
    destination_x = list(map(map_values_x, shots_by_type['X2']))
    origin_y = list(map(map_values_y, shots_by_type['Y']))
    destination_y = list(map(map_values_y, shots_by_type['Y2']))
    
    x = []
    y = []
    symbol = []
    
    for i in range(len(shots_by_type)):
        x.append(origin_x[i])
        x.append(destination_x[i])
        x.append(None)
        y.append(origin_y[i])
        y.append(destination_y[i])
        y.append(None)
        symbol.append('circle')
        symbol.append('arrow-right')
        symbol.append('arrow-right')
        
    
    color = assign_color_by_shot_type(shots_by_type['Event'])[0]
    
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers+lines',
        marker=dict(
            color=color,
            size=10,
            symbol=symbol
        ),
        # text=shots_by_type['Event'],
        name=shot_type,
        # visible='legendonly'
    ))

# Add figure title
fig.update_layout(
    showlegend=True,
    legend=dict(
        itemclick='toggle',
        itemdoubleclick='toggleothers'
    ),
    title_text= shots['Player'].iloc[0] 
                + " - Shots - Game 1" 
                + "<br><br> <sup>Click legend once to hide trace, double click legend to isolate trace!</sup>"
)

fig.show()

In [ ]:
# plot shots
fig = go.Figure()

# draw badminton court
draw_plotly_court(fig, show_title=True, labelticks=False, show_axis=False,
                  glayer='above', bg_color='black', margins=200)

shots = shots_team_1_player_1

for shot_type in shots['Event'].unique(): # add trace for every shot type
    
    shots_by_type = shots.loc[shots['Event'] == shot_type]
    
    # convert coordinates
    x = list(map(map_values_x, shots_by_type['X']))
    y = list(map(map_values_y, shots_by_type['Y']))
    
    color = assign_color_by_shot_type(shots_by_type['Event'])
    
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(
            color=color,
            size=10
        ),
        # text=shots_by_type['Event'],
        name=shot_type,
        # visible='legendonly'
    ))

# Add figure title
fig.update_layout(
    showlegend=True,
    legend=dict(
        itemclick='toggle',
        itemdoubleclick='toggleothers'
    ),
    title_text= shots['Player'].iloc[0] 
                + " - Shots Origin - Game 1" 
                + "<br><br> <sup>Click legend once to hide trace, double click legend to isolate trace!</sup>"
)

fig.show()

##